# Introduction to the NCAS CF Data Tools, cf-python and cf-plot

***

## Setting up

**In this section we set up this Notebook, import the libraries and check the data we will work with, ready to use the libraries within this notebook.**

Run some set up for nice outputs in this Jupyter Notebook (not required in interactive Python or a script):

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

Import cf-python and cf-plot:

In [ ]:
import cfplot as cfp
import cf

Inspect the versions of cf-python and cf-plot and the version of the CF Conventions those are matched to:

In [ ]:
print("cf-python version is:", cf.__version__)
print("cf-plot version is:", cfp.__version__)
print("CF Conventions version is:", cf.CF())

<div class="alert alert-block alert-info">
<i>Note:</i> you can work with data compliant by any other version of the CF Conventions, or without (much) compliance, but the CF Conventions version gives the maximum version that these versions of the tools understand the features of.
</div>

Finally, see what datasets we have to explore:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' preceeeds a shell command, so this is a terminal command and not Python
</div>

In [ ]:
!ls ../ncas_data

***

## 3. Reducing datasets by subspacing and collapsing

**In this section we show how multi-dimensional data can be tamed using cf-python so that you can get a reduced form that can be analysed or plotted, by reducing the dimensions by selecting a subset of point(s) along the axes or collapsing down according to some statistic such as the mean or an extrema.**

Often datasets represent highly multi-dimensional data, for example 4D or higher. Usually we want to find a either a sub-space, or a statistical representation (such as an average or extrema), of the full data array in less dimensions, such as in 3D or 2D or even in the form of a 1D time series or 0D statistic.

We'll demonstrate this with another dataset and field selected from it. This serves as a reminder on concepts from the first section of the Notebook:

<div class="alert alert-block alert-info">
<i>Note:</i> here we are reading in a file in the 'PP' format, which is a file format originating from the Met Office that can often be encountered in geoscience, like netCDF, hence the file extension '.pp'. You can read and process PP files using cf-python exactly the same way you do for netCDF files, so you do not need to concern yourself with the difference in file format in your code.
</div>

In [ ]:
field2 = cf.read("../ncas_data/aaaaoa.pmh8dec.pp")[2]
print(field2)

In this case, see the numbers representing axes sizes on the 'Data' line: we have a 3D field where the axes sizes are 17 for air pressure, 30 for grid latitude and 24 for grid latitude. To reduce this to a 2D form, we need to take one of the non-zero axes and convert it to size 1. This can be done either by **subspacing** or by **statistically collapsing** it.

### a) Subspacing using metadata conditions

Use the `subspace` method to find a subspace of a field, the output of which is another field reduced down in the way specified by the method arguments:

In [ ]:
field2_subspace1 = field2.subspace(air_pressure=1000.0000610351562)  # taking first value
print(field2_subspace1)

Let's do the same but to subspace to reduce the data along a different axis, this time `grid_latitude`:

In [ ]:
field2_subspace2 = field2.subspace(grid_latitude=-5.279999852180481)  # taking the last value
print(field2_subspace2)

### b) Subspacing using indexing, including equivalency to the above

We can also use indexing to do a subspace. So, instead of picking out a given value from the printed information,
we can use a specific Python index to pick it out.

The `air_pressure` coordinate is listed as first in axes order, so you must specify the index at the first position of the three i.e. `[<index>, :, :]` where `:` means to not take a subspace and leave the whole axis as it was. For example, the following takes the first (position 0 in Python indexing) value of the `air_pressure`:

In [ ]:
field2_subspace1_by_index = field2[0, :, :]  # taking first value from first coordinate
print(field2_subspace1_by_index)

To prove that this is the same field as we got using the direct `subspace` method from the sub-section above, namely `h.subspace(air_pressure=1000.0000610351562)`, we can use the `equals` method, which states whether one field is identical to another:

In [ ]:
field2_subspace1_by_index.equals(field2_subspace1)

Similarly, the `grid_latitude` coordinate is listed as second in axes order, so you must specify the index at the second position of the three i.e. `[:, <index>, :]`. Let's take the last (position -1 in Python indexing) value from this, like so:

In [ ]:
field2_subspace2_by_index = field2[:, -1, :]  # taking last value from second coordinate
print(field2_subspace2_by_index)

Again, to prove that this is the same field as we got as using the direct `subspace` approach previously with `h.subspace(grid_latitude=-5.279999852180481)`:

In [ ]:
field2_subspace2_by_index.equals(field2_subspace2)

You can do multiple subspaces at once via either of the methods above, so for example you can combine the two separate subspaces into one call:

In [ ]:
field2_subspace3 = field2.subspace(air_pressure=1000.0000610351562, grid_latitude=-5.279999852180481)
field2_subspace3_by_index = field2[0, -1, :]

This results in (in both cases):

In [ ]:
field2_subspace3.equals(field2_subspace3_by_index)
print(field2_subspace3)

### c) Statistical collapses

Instead of extracting the data at a particular value from the `air_pressure` dimension coordinate, we might want to collapse the data down according to a representative statistic covering all of those values, such as an average or extrema value. We do this with the `collapse` method.

Say we want to get the _mean_ of all of the air pressure values to reduce that coordinate from having the 17 values to just one mean representation, we would do this as follows:

In [ ]:
field2_collapse1 = field2.collapse("air_pressure: mean")  # taking mean of the 17 values for air pressure
print(field2_collapse1)

Another example is taking the _minimum_ of all of the values across the grid latitudes:

In [ ]:
field2_collapse2 = field2.collapse("grid_latitude: minimum")  # taking minimum of the 30 values for the grid latitude
print(field2_collapse2)

Equivalently, you can specify the axes via an `axes` argument when the coordinate to collapse along is one of `X`, `Y`, `Z` or `T`. So this `collapse` call will give the same result:

In [ ]:
field2_collapse3 = field2.collapse("minimum", axes="Y")  # taking minimum of the 30 values for the grid latitude
print(field2_collapse3)

Proving that this gives the same result as with the `"grid_latitude: minimum"` argument:

In [ ]:
field2_collapse3.equals(field2_collapse2)

***

## Where to find more information and resources on the NCAS CF Data Tools

Here are some links relating to the NCAS CF Data Tools and this training.

* This training, with further material, is hosted online and there are instructions for setting up the environment so you can work through it in your own time: https://github.com/NCAS-CMS/cf-tools-training.
* The cf-python documentation lives at https://ncas-cms.github.io/cf-python/.
* The cf-python code lives on GitHub at https://github.com/NCAS-CMS/cf-python. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-python/issues.
* The cf-plot documentation lives at https://ncas-cms.github.io/cf-plot/build/.
* The cf-plot code lives on GitHub at https://github.com/NCAS-CMS/cf-plot. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-plot/issues.
* There is a technical presentation about the NCAS CF Data Tools avaialble from https://hps.vi4io.org/_media/events/2020/summer-school-cfnetcdf.pdf.
* The website of the CF Conventions can be found at https://cfconventions.org/.
* The landing page for training into the CF Conventions is found here within the website above: https://cfconventions.org/Training/.

If you have any queries after this course, please either use the Issue Trackers linked above or you can email me at: sadie.bartholomew@ncas.ac.uk.

***